In [36]:
import numpy as np
import pandas as pd
import gurobipy as gp

In [37]:
# Reading the monthly ROI file.
monthly_roi = pd.read_csv('roi_mat.csv')
monthly_roi = monthly_roi.rename(columns={'Unnamed: 0': 'Months'})
display(monthly_roi)

# Reading the ROIs provided by the firms.
roi_firms = pd.read_csv('ROI_data.csv')
roi_firms = np.array(roi_firms.drop(['Platform'], axis=1))
roi_firms

,Months,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9
5,June,4.0,3.2,2.7,3.4,3.4,3.0,4.5,2.10,3.8,4.1
6,July,3.9,3.6,2.0,4.4,3.9,3.7,4.3,1.80,4.0,3.8
7,August,4.2,3.3,2.8,4.2,2.0,3.7,3.6,1.50,4.4,4.3
8,September,4.1,2.8,2.5,4.2,2.9,3.7,2.8,2.50,4.0,3.4
9,October,3.0,3.0,3.1,4.6,3.1,3.3,3.2,2.30,2.5,3.2


array([[0.031, 0.049, 0.024, 0.039, 0.016, 0.024, 0.046, 0.026, 0.033,
        0.044],
       [0.049, 0.023, 0.024, 0.039, 0.044, 0.046, 0.026, 0.019, 0.037,
        0.026]])

In [38]:
# Iterating over the ROIs received by the two independent firms, i.e, over "ROI_data.csv"
initial_budget = 10
for j, rois in enumerate(roi_firms):
    Mod = gp.Model()
    ModX = Mod.addMVar(10) # Initializing 10 variables for 10 marketing sources.

    # Creating the objective function.
    obj = 0
    for i in range(len(rois)):
        obj += (1 + rois[i])*ModX[i]
    
    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    # Adding Constraints
    Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= initial_budget) # Constarint for total budget
    Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9]) # Investment in Print and TV should be no more than the investment in Facebook and Email
    Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
    for i in range(10): # Adding constraint for investment in each segment less than $ 3M
        Mod.addConstr(ModX[i] <= 3)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    globals()["Allocation_"+str(j+1)] = ModX.X
    globals()["Objective_Value_"+str(j+1)] = Mod.objVal
    print("Allocation_"+str(j+1), ModX.X)
    print("Objective_Value_"+str(j+1), Mod.objVal)

Allocation_1 [0. 3. 0. 1. 0. 0. 3. 0. 0. 3.]
Objective_Value_1 10.456
Allocation_2 [3. 0. 0. 1. 3. 3. 0. 0. 0. 0.]
Objective_Value_2 10.456


In [39]:
diff_1 = Objective_Value_1 - (10 + np.dot(roi_firms[0], Allocation_2))
print("Using allocations from second iteration the objective of first iteration would decrease by ", diff_1)

diff_2 = Objective_Value_1 - (10 + np.dot(roi_firms[1], Allocation_1))
print("Using allocations from first iteration the objective of second iteration would decrease by ", diff_2)

Using allocations from second iteration the objective of first iteration would decrease by  0.20399999999999885
Using allocations from first iteration the objective of second iteration would decrease by  0.19200000000000017


In [40]:
# Assuming the first ROI as correct - performing a sensitivity analysis.
Mod = gp.Model()
ModX = Mod.addMVar(10) # Initializing 10 variables for 10 marketing sources.

# Creating the objective function.
obj = 0
for i in range(len(roi_firms[0])):
    obj += (1 + roi_firms[0][i])*ModX[i]

Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

# Adding Constraints
Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= initial_budget) # Constarint for total budget
Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9]) # Investment in Print and TV should be no more than the investment in Facebook and Email
Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
for i in range(10): # Adding constraint for investment in each segment less than $ 3M
    Mod.addConstr(ModX[i] <= 3)

Mod.Params.OutputFlag = 0
Mod.optimize()
obj_val = Mod.objVal
ModX.X

print("Lower Bound", ModX.SAObjLow)
print("Upper Bound", ModX.SAObjUp)

Lower Bound [ -inf 1.039  -inf 1.033  -inf  -inf 1.039  -inf  -inf 1.029]
Upper Bound [1.049 1.062 1.039 1.046 1.029 1.039   inf 1.039 1.039   inf]


In [41]:
obj_vals = []
obj_vals.append(initial_budget)
allocations = []
for i, month in enumerate(monthly_roi['Months']): # Iterating over each months ROI, assuming the initial investment to be $ 10M
    row = []
    row.append(month)
    rois = np.array(monthly_roi.drop('Months', axis=1))
    rois = rois[i]
    Mod = gp.Model()
    ModX = Mod.addMVar(10) # Initializing 10 variables for 10 marketing sources.

    # Creating the objective function.
    obj = 0
    for m in range(len(rois)):
        obj += (1 + (rois[m]/100))*ModX[m]

    Mod.setObjective(obj, sense=gp.GRB.MAXIMIZE)

    # Changing budget to reinvest 50% of the earnings each month.
    if i==0:
        budget = obj_vals[i]
    else:
        budget = obj_vals[i-1] + (obj_vals[i] - obj_vals[i-1]) / 2
    row.append(budget)

    # Adding Constraints
    Mod.addConstr(ModX[0] + ModX[1] + ModX[2] + ModX[3] + ModX[4] + ModX[5] + ModX[6] + ModX[7] + ModX[8] + ModX[9] <= budget) # Constarint for total budget
    Mod.addConstr(ModX[0] + ModX[1] <= ModX[4] + ModX[9]) # Investment in Print and TV should be no more than the investment in Facebook and Email
    Mod.addConstr(ModX[4] + ModX[5] + ModX[7] + ModX[6] + ModX[8] >= 2*(ModX[2] + ModX[3])) # Total amount in social media shoule be atleaset twice in SEO and AdWords
    for m in range(10): # Adding constraint for investment in each segment less than $ 3M
        Mod.addConstr(ModX[m] <= 3)

    Mod.Params.OutputFlag = 0
    Mod.optimize()
    
    for x in ModX.X:
        row.append(x)
    row.append(Mod.objVal)
    obj_vals.append(Mod.objVal)
    allocations.append(row)

columns=['Month', 'Budget', 'Print', 'TV', 'SEO', 'AdWords', 'Facebook', 'LinkedIn', 'Instagram', 'Snapchat', 'Twitter', 'Email', 'Obj_Value']
allocations = pd.DataFrame(allocations, columns=columns)
allocations.to_csv('allocations.csv', index=False)
display(allocations)

,Month,Budget,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email,Obj_Value
0,January,10.000000,3.000000,0.0,0.0,1.333333,0.000000,0.000000,2.666667,0.0,0.000000,3.00000,10.373000
1,February,10.186500,3.000000,0.0,0.0,2.395500,3.000000,0.000000,0.000000,0.0,1.791000,0.00000,10.592796
2,March,10.482898,0.000000,0.0,0.0,3.000000,0.000000,3.000000,1.482898,0.0,3.000000,0.00000,10.900765
3,April,10.746781,0.000000,0.0,0.0,3.000000,0.000000,3.000000,3.000000,0.0,1.746781,0.00000,11.166665
4,May,11.033715,2.033715,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,3.000000,3.00000,11.473895
5,June,11.320280,3.000000,0.0,0.0,0.000000,0.000000,0.000000,3.000000,0.0,2.320280,3.00000,11.786450
6,July,11.630173,1.315086,0.0,0.0,3.000000,1.315086,0.000000,3.000000,0.0,3.000000,0.00000,12.113749
7,August,11.950100,3.000000,0.0,0.0,1.983367,0.000000,0.966733,0.000000,0.0,3.000000,3.00000,12.456171
8,September,12.284960,1.642480,0.0,0.0,3.000000,0.000000,3.000000,0.000000,0.0,3.000000,1.64248,12.765146
9,October,12.610658,0.000000,0.0,0.0,3.000000,0.610658,3.000000,3.000000,0.0,0.000000,3.00000,13.058589
